In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.model_selection import train_test_split
import graphviz 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
%matplotlib inline

FDR@3% for training/testing/oot

In [2]:
data=pd.read_csv('0325_vars_final_zscale.csv')
oot=pd.read_csv('0325_oot_final_zscale.csv')
x = data.iloc[:,2:]
y = data.iloc[:,1]
x_oot = oot.iloc[:,2:]
y_oot = oot.iloc[:,1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
clf = MLPClassifier(hidden_layer_sizes=[5,5])
clf.fit(x_train, y_train)

NN_scores = clf.predict_proba(x_train)[:,1]
X_train=x_train
X_train['NN_scores']=NN_scores
Train=X_train.join(y_train)
Train=Train.sort_values(by=['NN_scores'],ascending=False)
GoodTest=sum(Train['Fraud']==0)
BadTest=sum(Train['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0

for i in range(100):
    data=Train.iloc[r:min((i+1)*630,62977),]
    populationbin=i+1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min((i+1)*630,62977)-r
    totalrecord=min((i+1)*630,62977)
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r=(i+1)*630
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
train_FDR=l.loc[2]['culperbad']


NN_scores = clf.predict_proba(x_test)[:,1]
X_test=x_test
X_test['NN_scores']=NN_scores
Test=X_test.join(y_test)
Test=Test.sort_values(by=['NN_scores'],ascending=False)
GoodTest=sum(Test['Fraud']==0)
BadTest=sum(Test['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0

for i in range(100):
    data=Test.iloc[r:min((i+1)*210,20993),]
    populationbin=i+1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min((i+1)*210,20993)-r
    totalrecord=min((i+1)*210,20993)
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r=(i+1)*210
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
test_FDR=l.loc[2]['culperbad']

NN_scores = clf.predict_proba(x_oot)[:,1]
X_oot=x_oot
X_oot['NN_scores']=NN_scores
OOT=X_oot.join(y_oot)
OOT=OOT.sort_values(by=['NN_scores'],ascending=False)
GoodTest=sum(OOT['Fraud']==0)
BadTest=sum(OOT['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0

for i in range(100):
    data=OOT.iloc[r:min((i+1)*125,12427),]
    populationbin=i+1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min((i+1)*125,12427)-r
    totalrecord=min((i+1)*125,12427)
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r=(i+1)*125
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
oot_FDR=l.loc[2]['culperbad']


print(train_FDR,test_FDR,oot_FDR)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


63.8109305760709 69.95073891625616 31.843575418994412


Training Table

In [3]:
data=pd.read_csv('0325_vars_final_zscale.csv')
oot=pd.read_csv('0325_oot_final_zscale.csv')
x = data.iloc[:,2:]
y = data.iloc[:,1]
x_oot = oot.iloc[:,2:]
y_oot = oot.iloc[:,1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
clf = MLPClassifier(hidden_layer_sizes=[5,5])
clf.fit(x_train, y_train)

NN_scores = clf.predict_proba(x_train)[:,1]
X_train=x_train
X_train['NN_scores']=NN_scores
Train=X_train.join(y_train)
Train=Train.sort_values(by=['NN_scores'],ascending=False)
GoodTest=sum(Train['Fraud']==0)
BadTest=sum(Train['Fraud']==1)

r=0
l=[]
culgood=0
culbad=0

for i in range(100):
    data=Train.iloc[r:min((i+1)*630,62977),]
    populationbin=i+1
    numofrecord=len(data)
    numofgood=sum(data['Fraud']==0)
    numofbads=sum(data['Fraud']==1)
    percentagegood=numofgood/GoodTest*100
    percentagebad=numofbads/BadTest*100
    numofrecord=min((i+1)*630,62977)-r
    totalrecord=min((i+1)*630,62977)
    culgood+=numofgood
    culbad+=numofbads
    culpergood=culgood/GoodTest*100
    culperbad=culbad/BadTest*100
    KS=culperbad-culpergood
    FPR=culpergood/culperbad
    r=(i+1)*630
    l.append([populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR])
    
l
l=pd.DataFrame(l,columns=['populationbin','numofrecord','numofgood','numofbads','percentagegood','percentagebad','totalrecord','culgood','culbad','culpergood','culperbad','KS','FPR'])
l.head(20)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,populationbin,numofrecord,numofgood,numofbads,percentagegood,percentagebad,totalrecord,culgood,culbad,culpergood,culperbad,KS,FPR
0,1,630,291,339,0.467095,50.073855,630,291,339,0.467095,50.073855,49.606761,0.009328
1,2,630,569,61,0.913323,9.010340,1260,860,400,1.380417,59.084195,57.703778,0.023364
2,3,630,593,37,0.951846,5.465288,1890,1453,437,2.332263,64.549483,62.217220,0.036131
3,4,630,602,28,0.966292,4.135894,2520,2055,465,3.298555,68.685377,65.386821,0.048024
4,5,630,615,15,0.987159,2.215657,3150,2670,480,4.285714,70.901034,66.615320,0.060446
5,6,630,615,15,0.987159,2.215657,3780,3285,495,5.272873,73.116691,67.843818,0.072116
6,7,630,618,12,0.991974,1.772526,4410,3903,507,6.264848,74.889217,68.624370,0.083655
7,8,630,615,15,0.987159,2.215657,5040,4518,522,7.252006,77.104874,69.852868,0.094054
8,9,630,621,9,0.996790,1.329394,5670,5139,531,8.248796,78.434269,70.185473,0.105168
9,10,630,623,7,1.000000,1.033973,6300,5762,538,9.248796,79.468242,70.219446,0.116384
